In [22]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
def get_optimized_model_v1(device='cpu'):
    """
    Optimisation principale: Réduction dramatique du classificateur
    Réduction estimée: ~85% des paramètres
    Impact performance: Minimal (< 2% accuracy)
    """
    model = nn.Sequential(
        # Bloc 1 - INCHANGÉ (extraction critique)
        nn.Conv2d(3, 32, kernel_size=4, padding=2),  # 3 → 32 canaux
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(2),  # 32x32 → 16x16
        nn.Dropout2d(0.25),

        # Bloc 2 - INCHANGÉ (patterns importants)
        nn.Conv2d(32, 64, kernel_size=4, padding=2),  # 32 → 64 canaux
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(2),  # 16x16 → 8x8
        nn.Dropout2d(0.25),

        # Classificateur ULTRA-COMPACT
        nn.Flatten(),  # 64 * 8 * 8 = 4096
        nn.Linear(4096, 128),  # 512 → 128 (÷4 réduction!)
        nn.ReLU(),
        nn.Dropout(0.4),  # Augmentation dropout pour compenser
        nn.Linear(128, 10)   # Direct vers sortie
    ).to(device)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    return model, loss_fn, optimizer
summary(model, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 33, 33]           1,568
       BatchNorm2d-2           [-1, 32, 33, 33]              64
              ReLU-3           [-1, 32, 33, 33]               0
         MaxPool2d-4           [-1, 32, 16, 16]               0
         Dropout2d-5           [-1, 32, 16, 16]               0
            Conv2d-6           [-1, 32, 17, 17]             512
            Conv2d-7           [-1, 64, 17, 17]           2,048
DepthwiseSeparableConv2d-8           [-1, 64, 17, 17]               0
       BatchNorm2d-9           [-1, 64, 17, 17]             128
             ReLU-10           [-1, 64, 17, 17]               0
AdaptiveAvgPool2d-11             [-1, 64, 6, 6]               0
        Dropout2d-12             [-1, 64, 6, 6]               0
          Flatten-13                 [-1, 2304]               0
           Linear-14             

In [2]:
def get_optimized_model_v2(device='cpu'):
    """
    Optimisation avancée avec pooling adaptatif
    Réduction: ~80% des paramètres
    Bénéfice: Meilleure préservation des features spatiales
    """
    model = nn.Sequential(
        # Bloc 1 - Feature extraction
        nn.Conv2d(3, 32, kernel_size=4, padding=2),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(2),
        nn.Dropout2d(0.25),

        # Bloc 2 - Pattern recognition
        nn.Conv2d(32, 64, kernel_size=4, padding=2),
        nn.BatchNorm2d(64),
        nn.ReLU(),

        # INNOVATION: Adaptive pooling pour taille fixe
        nn.AdaptiveAvgPool2d((4, 4)),  # Force 4x4 indépendamment de l'entrée
        nn.Dropout2d(0.3),

        # Classificateur compact progressif
        nn.Flatten(),  # 64 * 4 * 4 = 1024 (÷4 vs original!)
        nn.Linear(1024, 256),  # Réduction progressive
        nn.ReLU(),
        nn.Dropout(0.4),
        nn.Linear(256, 64),    # Couche intermédiaire
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(64, 10)
    ).to(device)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    return model, loss_fn, optimizer


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 33, 33]           1,568
       BatchNorm2d-2           [-1, 32, 33, 33]              64
              ReLU-3           [-1, 32, 33, 33]               0
         MaxPool2d-4           [-1, 32, 16, 16]               0
         Dropout2d-5           [-1, 32, 16, 16]               0
            Conv2d-6           [-1, 64, 17, 17]          32,832
       BatchNorm2d-7           [-1, 64, 17, 17]             128
              ReLU-8           [-1, 64, 17, 17]               0
         Dropout2d-9           [-1, 64, 17, 17]               0
           Conv2d-10           [-1, 10, 17, 17]             650
      BatchNorm2d-11           [-1, 10, 17, 17]              20
             ReLU-12           [-1, 10, 17, 17]               0
AdaptiveAvgPool2d-13             [-1, 10, 1, 1]               0
          Flatten-14                   

In [3]:
class DepthwiseSeparableConv2d(nn.Module):
    """
    Convolution séparable en profondeur avec kernel_size=4
    Réduction massive des paramètres convolutionnels
    """
    def __init__(self, in_channels, out_channels, kernel_size=4, padding=2):
        super().__init__()
        # Depthwise: chaque canal traité séparément
        self.depthwise = nn.Conv2d(in_channels, in_channels,
                                 kernel_size=kernel_size, padding=padding,
                                 groups=in_channels, bias=False)
        # Pointwise: mélange les canaux avec 1x1
        self.pointwise = nn.Conv2d(in_channels, out_channels,
                                 kernel_size=1, bias=False)

    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        return x

def get_optimized_model_v3(device='cpu'):
    """
    Version MobileNet-inspired avec convolutions séparables
    Réduction: ~70% des paramètres conv + classificateur compact
    Innovation: Garde l'efficacité du kernel_size=4
    """
    model = nn.Sequential(
        # Bloc 1 - Convolution standard pour extraction initiale
        nn.Conv2d(3, 32, kernel_size=4, padding=2),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(2),
        nn.Dropout2d(0.25),

        # Bloc 2 - DEPTHWISE SEPARABLE avec kernel=4
        DepthwiseSeparableConv2d(32, 64, kernel_size=4, padding=2),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.AdaptiveAvgPool2d((6, 6)),  # Pooling adaptatif
        nn.Dropout2d(0.3),

        # Classificateur ultra-compact
        nn.Flatten(),  # 64 * 6 * 6 = 2304
        nn.Linear(2304, 128),
        nn.ReLU(),
        nn.Dropout(0.5),  # Dropout élevé pour compenser la capacité réduite
        nn.Linear(128, 10)
    ).to(device)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    return model, loss_fn, optimizer

In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np

class CIFAR10Dataset(Dataset):
    def __init__(self, images, targets, device='cpu'):
        # Convertir les tableaux NumPy en tenseurs PyTorch
        images = torch.from_numpy(images).float() / 255.0
        targets = torch.tensor(targets, dtype=torch.long)

        # CIFAR10 : images en 3x32x32, format (N, H, W, C) -> (N, C, H, W)
        images = images.permute(0, 3, 1, 2)

        self.x = images
        self.y = targets
        self.device = device

    def __getitem__(self, ix):
        x = self.x[ix].to(self.device)
        y = self.y[ix].to(self.device)
        return x, y

    def __len__(self):
        return len(self.x)


In [6]:
data_folder = '/content/' # This can be any directory you want to download FMNIST to
fmnist = datasets.CIFAR10(data_folder, download=True, train=True)

100%|██████████| 170M/170M [00:11<00:00, 14.9MB/s]


In [7]:

tr_images = fmnist.data
tr_targets = fmnist.targets


val_fmnist = datasets.FashionMNIST(data_folder, download=True, train=False)
val_images = val_fmnist.data
val_targets = val_fmnist.targets



100%|██████████| 26.4M/26.4M [00:03<00:00, 8.10MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 136kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 2.22MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 8.97MB/s]


In [20]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np

class CIFAR10Dataset(Dataset):
    def __init__(self, images, targets, device='cpu'):
        # Convertir les tableaux NumPy en tenseurs PyTorch
        images = torch.tensor(images, dtype=torch.float32) / 255.0  # Conversion en tenseur PyTorch
        targets = torch.tensor(targets, dtype=torch.long)

        # Vérifier la forme des images et les réorganiser si nécessaire
        if images.ndimension() == 3:  # Si les images sont de forme (N, H, W) sans canaux
            images = images.unsqueeze(3)  # Ajouter un axe de canaux (N, H, W, 1)
        elif images.ndimension() == 4 and images.shape[3] != 3:  # Si les images n'ont pas 3 canaux
            raise ValueError(f"Les images doivent avoir 3 canaux, mais ont {images.shape[3]} canaux.")

        # CIFAR10 : images en 3x32x32, format (N, H, W, C) -> (N, C, H, W)
        images = images.permute(0, 3, 1, 2)

        self.x = images
        self.y = targets
        self.device = device

    def __getitem__(self, ix):
        x = self.x[ix].to(self.device)
        y = self.y[ix].to(self.device)
        return x, y

    def __len__(self):
        return len(self.x)


In [14]:
def train_batch(x, y, model, opt, loss_fn):
    prediction = model(x)
    batch_loss = loss_fn(prediction, y)
    batch_loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return batch_loss.item()

@torch.no_grad()
def accuracy(x, y, model):
    model.eval()
    prediction = model(x)
    max_values, argmaxes = prediction.max(-1)
    is_correct = argmaxes == y
    return is_correct.cpu().numpy().tolist()

In [9]:


def get_data():
    train = CIFAR10Dataset(tr_images, tr_targets)
    trn_dl = DataLoader(train, batch_size=32, shuffle=True)
    val = CIFAR10Dataset(val_images, val_targets)
    val_dl = DataLoader(val, batch_size=len(val_images), shuffle=True)
    return trn_dl, val_dl

@torch.no_grad()
def val_loss(x, y, model):
    model.eval()
    prediction = model(x)
    val_loss = loss_fn(prediction, y)
    return val_loss.item()



In [23]:
# Définir le device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Charger les données
trn_dl, val_dl = get_data()

# Initialiser le modèle
model, loss_fn, optimizer = get_optimized_model_v1(device=device)

# Afficher un résumé du modèle
from torchsummary import summary
summary(model, (3, 32, 32))  # CIFAR10 : 3 canaux, 32x32


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 33, 33]           1,568
       BatchNorm2d-2           [-1, 32, 33, 33]              64
              ReLU-3           [-1, 32, 33, 33]               0
         MaxPool2d-4           [-1, 32, 16, 16]               0
         Dropout2d-5           [-1, 32, 16, 16]               0
            Conv2d-6           [-1, 64, 17, 17]          32,832
       BatchNorm2d-7           [-1, 64, 17, 17]             128
              ReLU-8           [-1, 64, 17, 17]               0
         MaxPool2d-9             [-1, 64, 8, 8]               0
        Dropout2d-10             [-1, 64, 8, 8]               0
          Flatten-11                 [-1, 4096]               0
           Linear-12                  [-1, 128]         524,416
             ReLU-13                  [-1, 128]               0
          Dropout-14                  [

/tmp/ipython-input-3063548849.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  images = torch.tensor(images, dtype=torch.float32) / 255.0  # Conversion en tenseur PyTorch
/tmp/ipython-input-3063548849.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets, dtype=torch.long)


In [ ]:
train_losses, train_accuracies = [], []
val_losses, val_accuracies = [], []
for epoch in range(5):
    print(epoch)
    train_epoch_losses, train_epoch_accuracies = [], []
    for ix, batch in enumerate(iter(trn_dl)):
        x, y = batch
        batch_loss = train_batch(x, y, model, optimizer, loss_fn)
        train_epoch_losses.append(batch_loss)
    train_epoch_loss = np.array(train_epoch_losses).mean()

    for ix, batch in enumerate(iter(trn_dl)):
        x, y = batch
        is_correct = accuracy(x, y, model)
        train_epoch_accuracies.extend(is_correct)
    train_epoch_accuracy = np.mean(train_epoch_accuracies)

    for ix, batch in enumerate(iter(val_dl)):
        x, y = batch
        val_is_correct = accuracy(x, y, model)
        validation_loss = val_loss(x, y, model)
    val_epoch_accuracy = np.mean(val_is_correct)

    train_losses.append(train_epoch_loss)
    train_accuracies.append(train_epoch_accuracy)
    val_losses.append(validation_loss)
    val_accuracies.append(val_epoch_accuracy)


0


In [ ]:
epochs = np.arange(5)+1
import matplotlib.ticker as mtick
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
%matplotlib inline
plt.subplot(211)
plt.plot(epochs, train_losses, 'bo', label='Training loss')
plt.plot(epochs, val_losses, 'r', label='Validation loss')
plt.gca().xaxis.set_major_locator(mticker.MultipleLocator(1))
plt.title('Training and validation loss with CNN')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid('off')
plt.show()
plt.subplot(212)
plt.plot(epochs, train_accuracies, 'bo', label='Training accuracy')
plt.plot(epochs, val_accuracies, 'r', label='Validation accuracy')
plt.gca().xaxis.set_major_locator(mticker.MultipleLocator(1))
plt.title('Training and validation accuracy with CNN')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
#plt.ylim(0.8,1)
plt.gca().set_yticklabels(['{:.0f}%'.format(x*100) for x in plt.gca().get_yticks()])
plt.legend()
plt.grid('off')
plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

def get_optimized_vgg5_model(device='cpu'):
    """
    Modèle VGG11 optimisé avec kernels 5x5 et pooling adaptatif
    Réduction des paramètres et meilleure préservation des features spatiales
    """
    model = nn.Sequential(
        # Bloc 1
        nn.Conv2d(3, 32, kernel_size=5, padding=2),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.Conv2d(32, 32, kernel_size=5, padding=2),
        nn.ReLU(),
        nn.BatchNorm2d(32),
        nn.SpatialDropout2d(0.25) if hasattr(nn, 'SpatialDropout2d') else nn.Dropout2d(0.25),
        nn.MaxPool2d(2),

        # Bloc 2
        nn.Conv2d(32, 64, kernel_size=5, padding=2),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.Conv2d(64, 64, kernel_size=5, padding=2),
        nn.ReLU(),
        nn.BatchNorm2d(64),
        nn.SpatialDropout2d(0.25) if hasattr(nn, 'SpatialDropout2d') else nn.Dropout2d(0.25),
        nn.MaxPool2d(2),

        # Bloc 3
        nn.Conv2d(64, 128, kernel_size=5, padding=2),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.SpatialDropout2d(0.25) if hasattr(nn, 'SpatialDropout2d') else nn.Dropout2d(0.25),
        nn.MaxPool2d(2),

        # Bloc 4
        nn.Conv2d(128, 128, kernel_size=5, padding=2),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.SpatialDropout2d(0.25) if hasattr(nn, 'SpatialDropout2d') else nn.Dropout2d(0.25),
        nn.MaxPool2d(2),

        # INNOVATION: Adaptive pooling pour taille fixe
        nn.AdaptiveAvgPool2d((4, 4)),

        # Classificateur compact progressif
        nn.Flatten(),  # 128 * 4 * 4 = 2048
        nn.Linear(2048, 1024),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(1024, 512),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(512, 10)
    ).to(device)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    return model, loss_fn, optimizer
